## III - Неопределенность оптимального портфеля. Оптимизация CVaR. 

Целью работы является оценка неопределенности оптимального портфеля для нормального многомерного распределения и двух способов вычисления оптимального портфеля (оптимизация в модели Марковица и оптимизация CVaR)

### Подготовка модели

#### Загружаем тикеры с фоднового рынка NASDAQ

In [109]:
import pandas as pd
import numpy as np

DATA_PATH    = 'Documents/GitHub/labs/fin-ops/downloader-data'
TICKERS_PATH = 'Documents/GitHub/labs/fin-ops/tickers'

def read_tickers(stock_markets) -> pd.DataFrame:
    '''Returns pandas dataframe containing all tickers for the @stock_markets '''
    ticker_files = [f'{TICKERS_PATH}/{sm}.csv' for sm in stock_markets]
    tickers = pd.concat([pd.read_csv(tf) for tf in ticker_files], ignore_index=True)
    return tickers


stock_markets = ['NASDAQ']
tickers = read_tickers(stock_markets)
tickers

,ticker,company
0,AAIT,iShares MSCI All Country Asia Information Tech...
1,AAL,"American Airlines Group, Inc."
2,AAME,Atlantic American Corporation
3,AAOI,"Applied Optoelectronics, Inc."
4,AAON,"AAON, Inc."
...,...,...
2962,ZN,Zion Oil & Gas Inc
2963,ZNGA,Zynga Inc.
2964,ZSPH,"ZS Pharma, Inc."
2965,ZU,"zulily, inc."


#### Загружаем исторические данные за 2021 год для полученных тикеров

In [110]:
def read_historical_data(tickers):
    '''Returns dict {ticker : historical data for the ticker}'''
    data_for_ticker = {}
    for index, (ticker, name) in tickers.iterrows():
        try:
            data = pd.read_csv(f'{DATA_PATH}/{ticker}.csv')
            if len(data) > 100:
                data_for_ticker[ticker] = data
        except:
            pass
    return data_for_ticker


data_for_ticker = read_historical_data(tickers)

#### Пример данных для AAPL:

In [111]:
data_for_ticker['AAPL']

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2020-11-30,116.597423,120.584682,116.437929,118.670799,169410200,0.0,0
1,2020-12-01,120.624557,123.076720,119.627742,122.329109,127728200,0.0,0
2,2020-12-02,121.631329,122.977035,120.504931,122.687958,89004200,0.0,0
3,2020-12-03,123.126555,123.385729,121.820730,122.548409,78967600,0.0,0
4,2020-12-04,122.209495,122.468669,121.132933,121.860611,78260400,0.0,0
...,...,...,...,...,...,...,...,...
248,2021-11-23,161.119995,161.800003,159.059998,161.410004,96041900,0.0,0
249,2021-11-24,160.750000,162.139999,159.639999,161.940002,69463600,0.0,0
250,2021-11-26,159.570007,160.449997,156.360001,156.809998,76959800,0.0,0
251,2021-11-29,159.369995,161.190002,158.789993,160.240005,88748200,0.0,0


#### Выбираем 20 активов

In [112]:
assets = ['MDLZ', 'MSFT', 'NXPI', 'PCAR', 'INTC',
          'NVDA', 'ILMN', 'DXCM', 'ROST', 'LULU',
          'PRFT', 'FTNT', 'WIRE', 'NFLX', 'GAIN',
          'SYNA', 'NYMTP', 'ANAT', 'TRNS', 'SFBS']

assets_data = {}
for asset in assets:
    assets_data[asset] = data_for_ticker[asset]

#### Оцениваем математические ожидания доходностей

In [113]:
import pandas as pd
import numpy as np

def add_logret(ticker_data: pd.DataFrame, by_column='Close') -> pd.DataFrame:
    '''Returns @ticker_data with calculated "logret" and "-logret" columns'''
    ticker_data = ticker_data.assign(logret=np.log(ticker_data[by_column]).diff())
    ticker_data['-logret'] = ticker_data['logret'].mul(-1)
    return ticker_data


def get_logret_mean_std(tickers, data_map, by_column='Close') -> pd.DataFrame:
    '''Returns @result pd.DataFrame such that @result.loc[ticker] == [logret_mean, logret_std]'''
    result = pd.DataFrame(data=[], columns=['ticker', 'logret_mean', 'logret_std'])
    result.set_index('ticker', inplace=True)

    for ticker in tickers:
        ticker_data = data_map[ticker]
        ticker_logret = np.log(ticker_data[by_column]).diff()
        result.loc[ticker] = [ticker_logret.mean(), ticker_logret.std()]
    
    return result
  

for ticker in assets_data.keys():
    assets_data[ticker] = add_logret(assets_data[ticker])

estims = get_logret_mean_std(assets, assets_data)
estims

,logret_mean,logret_std
ticker,,
MDLZ,0.000164,0.009635
MSFT,0.001743,0.012638
NXPI,0.001406,0.023336
PCAR,-0.000078,0.015266
INTC,0.000118,0.020613
NVDA,0.003541,0.025897
ILMN,0.000500,0.020907
DXCM,0.002243,0.022329
ROST,0.000076,0.018766


#### Находим матрицу выборочных ковариаций доходностей 

In [114]:
def get_covariation_matrix(tickers,                          
                           data_map=data_for_ticker,
                           by_column='logret'):
    columns = pd.DataFrame()
    for ticker in tickers:
        columns[ticker] = data_map[ticker][by_column][1:]
        
    matrix = columns.corr()
    return matrix

cov_matrix = get_covariation_matrix(assets, data_map=assets_data)
cov_matrix

,MDLZ,MSFT,NXPI,PCAR,INTC,NVDA,ILMN,DXCM,ROST,LULU,PRFT,FTNT,WIRE,NFLX,GAIN,SYNA,NYMTP,ANAT,TRNS,SFBS
MDLZ,1.000000,0.260625,0.136202,0.166086,0.128461,0.071711,0.106817,0.121394,0.320069,0.200534,0.142022,0.184870,0.108808,0.183858,0.145688,0.074572,-0.016072,0.090393,0.076097,0.188254
MSFT,0.260625,1.000000,0.435936,0.129668,0.343282,0.563342,0.365982,0.396934,0.237147,0.369962,0.340569,0.520006,0.156858,0.460941,0.266493,0.374816,0.088876,-0.054785,0.171884,0.002140
NXPI,0.136202,0.435936,1.000000,0.314621,0.518692,0.567564,0.348801,0.352195,0.347588,0.422580,0.419310,0.465200,0.366023,0.232486,0.300793,0.576943,0.114702,0.194865,0.201715,0.371353
PCAR,0.166086,0.129668,0.314621,1.000000,0.285791,0.146135,0.077887,0.024713,0.310914,0.112148,0.185204,0.087999,0.327231,-0.011242,0.357433,0.209517,0.091692,0.189116,0.160347,0.414896
INTC,0.128461,0.343282,0.518692,0.285791,1.000000,0.352533,0.238309,0.220853,0.287966,0.255882,0.231893,0.245138,0.141588,0.236885,0.280478,0.366278,0.132862,0.064409,0.115456,0.216980
NVDA,0.071711,0.563342,0.567564,0.146135,0.352533,1.000000,0.296262,0.351007,0.154648,0.415916,0.425477,0.490961,0.093375,0.356414,0.243640,0.441977,0.087635,-0.070881,0.113711,-0.013629
ILMN,0.106817,0.365982,0.348801,0.077887,0.238309,0.296262,1.000000,0.433226,0.030674,0.326248,0.271251,0.369235,0.078505,0.249747,0.162399,0.236065,0.031524,0.025273,0.094362,0.011555
DXCM,0.121394,0.396934,0.352195,0.024713,0.220853,0.351007,0.433226,1.000000,0.161316,0.370966,0.193181,0.345706,0.067249,0.343953,0.184725,0.343466,-0.024543,0.041092,0.198570,0.015732
ROST,0.320069,0.237147,0.347588,0.310914,0.287966,0.154648,0.030674,0.161316,1.000000,0.287834,0.314949,0.218192,0.409139,0.082407,0.331344,0.318590,-0.018868,0.234797,0.131619,0.430977
LULU,0.200534,0.369962,0.422580,0.112148,0.255882,0.415916,0.326248,0.370966,0.287834,1.000000,0.315082,0.393846,0.156097,0.306344,0.189652,0.252620,-0.010867,0.026117,0.062358,0.081343


In [115]:
import plotly.express as px

fig = px.imshow(cov_matrix, title='Correlation Matrix',
                color_continuous_scale=px.colors.diverging.RdYlGn[::-1])
fig.show()

#### Проверяем вырожденность матрицы и число обусловленности

In [116]:
det = np.linalg.det(cov_matrix)
print(f'det = {det}')

con = np.linalg.cond(cov_matrix, p='fro')
print(f'con = {con}')

det = 0.002219356005352488
con = 55.688725974909225


### 1. Истинный оптимальный портфель в модели Марковица с заданным отношением к риску. 

Находим константу с:

$$ с = \Phi^{-1}(\beta)$$

In [117]:
import math
from scipy.stats import norm

beta = 0.95
с = (norm.ppf(beta))**(-1)
print(f'с = {с}')

с = 0.6079568319117692


#### Решаем задачу оптимизации 

$$ -E(x)+ с \cdot σ(x) \rightarrow min $$
при условиях:
$$ x_1 + x_2 + ... +x_N = 1 $$
$$ x_i >= 0 $$
где
$$ E(x)= E_1x_1 + E_2 x_2 + ... + E_Nx_N $$
$$ σ^2(x)=\Sigma\Sigma σ_{i,j} x_i \cdot x_j $$

In [118]:
import math 
import numpy as np
from scipy.optimize import minimize

def get_E(x, vector_E):
    E = sum([(E_i * x_i) for E_i, x_i in zip(vector_E, x)]) 
    return E


def get_sigma(x, matrix_cov):
    sigma_squared = 0
    
    for i in range(len(x)):
        for j in range(len(x)):
            simga_i_j = matrix_cov.iloc[i].iloc[j]
            sigma_squared += simga_i_j * x[i] * x[j]
    
    sigma = math.sqrt(sigma_squared)
    return sigma
            
            
def target_function(x, vector_E, matrix_cov):
    E = get_E(x, vector_E)
    sigma = get_sigma(x, matrix_cov)
    result = -E + с * sigma
    return result


def find_optimal(vector_E=estims['logret_mean'], matrix_cov=cov_matrix):
    x0 = np.array([1/len(assets)] * len(assets))
    solution = minimize(target_function, x0, args=(vector_E, matrix_cov), 
                        method='SLSQP', 
                        constraints=[{'type': 'eq',  'fun': lambda x: sum(x) - 1}],
                        bounds=[(0, 1)] * len(assets))
    if not solution.success:
        raise Exception(opt.message)
    return solution
    
true_opt_solution = find_optimal()

#### Веса портфеля:

In [119]:
import plotly.express as px

print(f'Сумма весов: {sum(true_opt_solution.x)}')
fig = px.bar(x=assets, y=true_opt_solution.x)
fig.show()

Сумма весов: 0.9999999999999999


#### Значение целевой функции:

In [120]:
print(true_opt_solution.fun)

0.2624014886357103


### 2. Оценка неопределенности оптимального портфеля в модели Марковица с заданным отн. к риску. 

#### 2.1 Задаём число наблюдений T=30. С помощью генератора многомерного нормального распределения создаём выборку размера Т из нормального распределения с вектором математических ожиданий  E=(E1, E2, …, EN) и матрицей ковариаций (σi,j). 

In [121]:
T = 30
sample_raw = np.random.multivariate_normal(estims['logret_mean'], cov_matrix, T)
sample = pd.DataFrame(columns=assets, data=sample_raw)
sample

,MDLZ,MSFT,NXPI,PCAR,INTC,NVDA,ILMN,DXCM,ROST,LULU,PRFT,FTNT,WIRE,NFLX,GAIN,SYNA,NYMTP,ANAT,TRNS,SFBS
0,-0.627235,-2.696205,-0.236222,-0.993358,-0.585076,-1.861673,-0.615184,-0.354867,-0.248533,-1.714950,-1.239607,-1.420831,-1.367469,-1.970434,-2.036409,-1.493213,-0.396817,0.480171,-0.503270,0.341944
1,0.047153,0.575098,-1.920844,-1.427491,-1.370342,-0.463954,-1.027600,-0.636766,-0.053583,-1.798055,-0.835974,-1.988900,0.147973,0.865198,-1.104330,-0.611693,0.149232,-1.736447,0.273499,-0.868628
2,0.550195,0.926836,0.695616,-0.054431,0.371789,0.153989,0.460374,1.071474,0.691710,0.774287,0.967844,0.854714,0.648789,1.018313,-1.587213,-0.041203,-2.067031,0.263773,0.631462,0.672410
3,0.717426,-0.122339,0.393722,3.275798,-0.044277,0.831878,-1.574107,-0.818764,0.757507,0.085336,1.567446,-0.072257,1.541345,-0.789956,2.222876,1.378341,1.136519,0.619370,0.337246,0.588897
4,0.847346,1.591711,0.691705,-0.493940,0.693623,1.186289,1.332957,0.996069,0.292041,0.163742,-0.232307,0.611165,-0.234855,0.235335,1.274978,-0.240999,1.013698,1.131040,1.303682,-1.045474
5,-2.098856,-2.376235,-1.694852,0.701301,-1.050432,-0.646040,0.302559,0.813228,1.231665,0.068127,-0.649822,-1.354894,0.792338,-0.471953,0.641294,-0.687950,-0.776678,1.660724,1.745020,0.697634
6,-1.146592,1.657685,1.942332,1.994793,0.340868,2.332664,1.825349,1.227026,-0.432205,1.291522,1.374258,1.850300,1.403639,0.525273,1.283443,2.142445,1.799885,1.089061,-0.060740,0.843985
7,0.603169,0.419405,1.115686,-1.905140,-0.508409,1.425505,1.009655,0.934455,0.837551,1.515651,0.871834,1.190555,0.163270,-0.963030,-0.876331,0.866669,-0.882805,0.533706,-0.110640,-0.601172
8,0.363336,-1.531169,-0.508107,-2.069387,-0.807192,0.503783,-1.223828,-1.169091,-2.224205,-0.757733,-0.912313,-1.683208,-0.032907,-0.476703,-0.113340,0.253386,0.855771,-0.733822,1.010602,-0.801558
9,0.139510,-0.133405,-0.199211,0.545375,-0.591656,0.393758,0.360618,1.999389,-2.113711,-1.533998,0.383784,-0.447090,-0.227559,-0.669046,-1.110637,0.307033,-0.355119,-1.108483,2.365440,-0.113780


#### 2.2 По построенной выборке делаем оценку Eest вектора математических ожиданий ...

In [122]:
estE = sample.mean()
estE

MDLZ    -0.038947
MSFT    -0.021063
NXPI    -0.306006
PCAR    -0.215617
INTC    -0.255713
NVDA     0.081914
ILMN    -0.051087
DXCM    -0.035644
ROST     0.032529
LULU    -0.062379
PRFT     0.011542
FTNT    -0.086381
WIRE    -0.057177
NFLX     0.057470
GAIN    -0.113643
SYNA    -0.078835
NYMTP    0.044523
ANAT    -0.155053
TRNS     0.186870
SFBS    -0.160529
dtype: float64

#### ... и оценку (σesti,j) матрицы ковариаций. 

In [123]:
estCov = sample.corr()
estCov

,MDLZ,MSFT,NXPI,PCAR,INTC,NVDA,ILMN,DXCM,ROST,LULU,PRFT,FTNT,WIRE,NFLX,GAIN,SYNA,NYMTP,ANAT,TRNS,SFBS
MDLZ,1.000000,0.484394,0.108162,-0.011557,0.094445,0.257004,0.047843,0.110541,0.287987,0.381246,0.447187,0.306702,0.052157,0.283376,0.291273,0.158549,-0.065786,-0.035961,0.193361,0.131271
MSFT,0.484394,1.000000,0.384935,0.023687,0.399469,0.638039,0.568916,0.343025,0.328390,0.522046,0.549409,0.761975,0.203952,0.602605,0.198767,0.253612,0.252754,-0.141428,-0.032538,-0.142309
NXPI,0.108162,0.384935,1.000000,0.339197,0.643329,0.624239,0.470778,0.322613,0.053697,0.403862,0.500476,0.456599,0.275316,0.105243,0.195731,0.577232,0.257153,0.144766,-0.006814,0.244496
PCAR,-0.011557,0.023687,0.339197,1.000000,0.410015,0.208781,-0.070431,-0.070351,0.182665,0.182100,0.414904,-0.012870,0.374608,-0.147744,0.532052,0.541629,0.094666,0.229353,0.078605,0.609444
INTC,0.094445,0.399469,0.643329,0.410015,1.000000,0.398309,0.387652,0.152732,0.368373,0.459935,0.311697,0.414877,0.250697,0.071348,0.457021,0.402029,0.276633,-0.130701,-0.003312,0.182272
NVDA,0.257004,0.638039,0.624239,0.208781,0.398309,1.000000,0.442922,0.334673,0.055493,0.504089,0.551419,0.624704,0.344305,0.343692,0.204569,0.640720,0.511203,0.043695,0.116852,-0.057290
ILMN,0.047843,0.568916,0.470778,-0.070431,0.387652,0.442922,1.000000,0.692296,0.107888,0.489470,0.457815,0.706338,0.024287,0.227206,0.090409,0.159638,0.118784,0.251903,0.066602,-0.023892
DXCM,0.110541,0.343025,0.322613,-0.070351,0.152732,0.334673,0.692296,1.000000,-0.047689,0.259949,0.320913,0.512944,-0.171508,0.279647,0.080180,0.178550,-0.130480,0.211505,0.184829,0.011351
ROST,0.287987,0.328390,0.053697,0.182665,0.368373,0.055493,0.107888,-0.047689,1.000000,0.449379,0.224010,0.290252,0.316487,0.145370,0.386814,0.086531,-0.019597,0.048742,-0.096656,0.282513
LULU,0.381246,0.522046,0.403862,0.182100,0.459935,0.504089,0.489470,0.259949,0.449379,1.000000,0.428039,0.669768,0.282949,0.347812,0.559578,0.389287,0.098313,0.038968,0.034662,0.127153


In [124]:
import plotly.express as px

fig = px.imshow(estCov, title='Correlation Matrix',
                color_continuous_scale=px.colors.diverging.RdYlGn[::-1])
fig.show()

#### 2.3 Используя эти оценки решаем задачу оптимизации 

In [125]:
opt_solution = find_optimal(estE, estCov)

#### Находим и фиксируем веса портфеля ...

In [126]:
print(f'Сумма весов: {sum(opt_solution.x)}')
fig = px.bar(x=assets, y=opt_solution.x)
fig.show()

Сумма весов: 1.0000000000000004


#### и значение целевой функции

In [127]:
print(opt_solution.fun)

0.197984272155834


#### 2.4 Сравниаем два портфеля: истинный (п.1) и выборочный (п.2.3). Оцениваем относительную ошибку в определении весов портфеля в норме Manhattan (L1 норма Минковского). Делайте выводы и сравнение в системе координат (σ, E). 

In [128]:
from scipy.spatial.distance import cityblock
cityblock(true_opt_solution.x, opt_solution.x)

1.1158336449343775

#### 2.5 Повторите эксперимент S=40 раз и оцените среднюю относительную ошибку по S повторениям эксперимента. Сделайте выводы.  Сделайте сравнение в системе координат (σ, E). 

In [129]:
import copy

S = 40
errors = []
x_vectors = pd.DataFrame(columns = [f'x_{i}' for i in range(len(assets))], data=[])


for iteration in range(0, S):
    T = 30
    sample_raw = np.random.multivariate_normal(estims['logret_mean'], cov_matrix, T)
    sample = pd.DataFrame(columns=assets, data=sample_raw)
    
    estE = sample.mean()
    estCov = sample.cov()
    
    opt_solution = find_optimal(estE, estCov)
        
    errors.append(cityblock(true_opt_solution.x, opt_solution.x))
    x_vectors.loc[iteration] = (copy.deepcopy(opt_solution.x))

    
print(f'Mean Erorr = {np.mean(errors)}')
print(f'Mean X:')
x_vectors.mean()

Mean Erorr = 1.2931801291121534
Mean X:


x_0     0.073060
x_1     0.058124
x_2     0.006085
x_3     0.046647
x_4     0.033299
x_5     0.057037
x_6     0.076515
x_7     0.035264
x_8     0.021175
x_9     0.044473
x_10    0.048252
x_11    0.027472
x_12    0.053902
x_13    0.062691
x_14    0.018401
x_15    0.023723
x_16    0.122499
x_17    0.056028
x_18    0.069585
x_19    0.065768
dtype: float64

In [130]:
import plotly.express as px

fig = px.bar(x=assets, y=x_vectors.mean(), title='Mean X')
fig.show()

#### 2.6  Предположите, что нам известны точные значения математических ожиданий E=(E1, E2, …, EN). Повторяем пп. 2.2-2.5. используя оценку только матрицы ковариаций

In [131]:
import copy

S = 40
errors = []
x_vectors = pd.DataFrame(columns = [f'x_{i}' for i in range(len(assets))], data=[])


for iteration in range(0, S):
    T = 30
    sample_raw = np.random.multivariate_normal(estims['logret_mean'], cov_matrix, T)
    sample = pd.DataFrame(columns=assets, data=sample_raw)
    
    estE = sample.mean()
    estCov = sample.cov()
    
    opt_solution = find_optimal(estims['logret_mean'], estCov)
        
    errors.append(cityblock(true_opt_solution.x, opt_solution.x))
    x_vectors.loc[iteration] = (copy.deepcopy(opt_solution.x))

    
print(f'Mean Erorr = {np.mean(errors)}')
print(f'Mean X:')
x_vectors.mean()

Mean Erorr = 0.8892722881137576
Mean X:


x_0     0.080107
x_1     0.012843
x_2     0.001826
x_3     0.051800
x_4     0.045563
x_5     0.041645
x_6     0.054977
x_7     0.040473
x_8     0.020464
x_9     0.023342
x_10    0.030938
x_11    0.046346
x_12    0.051800
x_13    0.065017
x_14    0.010234
x_15    0.019491
x_16    0.174925
x_17    0.125652
x_18    0.065881
x_19    0.036675
dtype: float64

In [132]:
import plotly.express as px

fig = px.bar(x=assets, y=x_vectors.mean(), title='Mean X')
fig.show()

### 3. Оценка неопределенности оптимального CVaR портфеля

С заданным отношением к риску  подобираем константу b таким образом, что истинный оптимальный CVaR портфель совпадает с истинным оптимальным портфелем п.1. Значение константы смотри в упражнениях к теме.

$$ b = \frac{1}{\sqrt{2 \pi }} \frac{1}{(1 - \beta)} exp(-(\Phi^{-1}(\beta))^2 / 2)$$

In [133]:
beta = 0.95
b = (1 / math.sqrt(2 * math.pi)) * (1 / (1 - beta)) * np.exp(-((norm.ppf(beta))**(-1))**2 / 2) 
print(f'b = {b}')

b = 6.632540979594675


In [134]:
def get_E(x, vector_E):
    E = sum([(E_i * x_i) for E_i, x_i in zip(vector_E, x)]) 
    return E


def get_sigma(x, matrix_cov):
    sigma_squared = 0
    
    for i in range(len(x)):
        for j in range(len(x)):
            simga_i_j = matrix_cov.iloc[i].iloc[j]
            sigma_squared += simga_i_j * x[i] * x[j]
    
    sigma = math.sqrt(sigma_squared)
    return sigma
            
            
def target_function_CVaR(x, vector_E, matrix_cov):
    E = get_E(x, vector_E)
    sigma = get_sigma(x, matrix_cov)
    result = -E + b * sigma
    return result


def find_optimal_CVaR(vector_E, matrix_cov):
    x0 = np.array([1/len(assets)] * len(assets))
    solution = minimize(target_function_CVaR, x0, args=(vector_E, matrix_cov), 
                        method='SLSQP', 
                        constraints=[{'type': 'eq',  'fun': lambda x: sum(x) - 1}],
                        bounds=[(0, 1)] * len(assets))
    if not solution.success:
        raise Exception(opt.message)
    return solution
    

#### Используя оценки из 2.2 решаем задачу оптимизации 

In [135]:
opt_solution_CVaR = find_optimal_CVaR(estE, estCov)

#### Находим и фиксируем веса портфеля ...

In [136]:
print(f'Сумма весов: {sum(opt_solution_CVaR.x)}')
fig = px.bar(x=assets, y=opt_solution_CVaR.x)
fig.show()

Сумма весов: 1.0000000000000062


#### и значение целевой функции

In [137]:
print(opt_solution_CVaR.fun)

1.784027703794713


#### 3.2 Сравниаем два портфеля: истинный (п.1) и найденный в п.3.1. Оцениваем относительную ошибку в определении весов портфеля в норме Manhattan (L1 норма Минковского). Сравниваем с ошибкой портфеля из п. 2.3

In [138]:
er1 = cityblock(true_opt_solution.x, opt_solution.x)
er2 = cityblock(true_opt_solution.x, opt_solution_CVaR.x)
print(f'Ошибка оптимального VaR портфеля(п.2.3) = {er1}')
print(f'Ошибка оптимального CVaR портфеля = {er2}')

Ошибка оптимального VaR портфеля(п.2.3) = 0.7467023553709373
Ошибка оптимального CVaR портфеля = 0.7782452616953123


#### 3.3 Повторите эксперимент S=40 раз и оцените среднюю относительную ошибку по S повторениям эксперимента. Сделайте выводы.  Сравните с ошибкой из п. 2.5.

In [139]:
import copy

S = 40
errors = []
x_vectors = pd.DataFrame(columns = [f'x_{i}' for i in range(len(assets))], data=[])


for iteration in range(0, S):
    T = 30
    sample_raw = np.random.multivariate_normal(estims['logret_mean'], cov_matrix, T)
    sample = pd.DataFrame(columns=assets, data=sample_raw)
    
    estE = sample.mean()
    estCov = sample.cov()
    
    opt_solution_CVaR = find_optimal_CVaR(estE, estCov)
        
    errors.append(cityblock(true_opt_solution.x, opt_solution_CVaR.x))
    x_vectors.loc[iteration] = (copy.deepcopy(opt_solution_CVaR.x))

    
print(f'Mean Erorr = {np.mean(errors)}')
print(f'Mean X:')
x_vectors.mean()

Mean Erorr = 0.9039719165206025
Mean X:


x_0     0.096962
x_1     0.019852
x_2     0.002625
x_3     0.053974
x_4     0.032394
x_5     0.040931
x_6     0.068270
x_7     0.043313
x_8     0.037849
x_9     0.014848
x_10    0.012367
x_11    0.023799
x_12    0.035153
x_13    0.081930
x_14    0.032451
x_15    0.039013
x_16    0.160324
x_17    0.095558
x_18    0.060715
x_19    0.047672
dtype: float64

In [140]:
import plotly.express as px

fig = px.bar(x=assets, y=x_vectors.mean(), title='Mean X')
fig.show()